이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

     |████████████████████████████████| 199 kB 5.8 MB/s eta 0:00:01
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyspark 3.4.0 requires py4j==0.10.9.7, but you have py4j 0.10.9.5 which is incompatible.


**Spark Session**을 하나 만든다

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark FS Partition Demo") \
        .master("spark://spark-master:7077") \
        .config("spark.executor.memory", "512m")\
        .enableHiveSupport() \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/04 05:27:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2023-09-25 12:29:30--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 52.92.209.10, 52.92.149.66, 3.5.77.142, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|52.92.209.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv.1’

appl_stock.csv.1    100%[===================>] 139.78K   501KB/s    in 0.3s    

2023-09-25 12:29:31 (501 KB/s) - ‘appl_stock.csv.1’ saved [143130/143130]



In [2]:
!ls

PySpark_FileSystem_Partitioning.ipynb  appl_stock.csv
PySpark_ML_실습1.ipynb		       bucketDemo.py
PySpark_ML_실습2.ipynb		       derby.log
PySpark_Schema_Evolution.ipynb	       join_broadcast_join
PySpark_Write_실습.ipynb	       wordcount


In [2]:
df = spark.read.csv("appl_stock.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [4]:
df.show(10)

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [4]:
df = df.withColumn("year", year(df.Date)) \
    .withColumn("month", month(df.Date))

df.show(10)

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+----+-----+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|year|month|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+----+-----+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|    1|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|    1|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|    1|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|2010|    1|
|2010-01-08 00:00:00|        210.2

In [5]:
spark.sql("DROP TABLE IF EXISTS appl_stock")

23/10/04 05:28:59 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/10/04 05:28:59 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/10/04 05:29:00 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/10/04 05:29:00 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.21.0.2
23/10/04 05:29:00 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [6]:
df.write.partitionBy("year", "month").saveAsTable("appl_stock")

23/10/04 05:29:21 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/10/04 05:29:21 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
23/10/04 05:29:21 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/10/04 05:29:21 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [8]:
!ls -tl 

total 412
drwxr-xr-x 3 root root     96 Sep 25 11:57 spark-warehouse
drwxr-xr-x 9 root root    288 Sep 25 11:56 metastore_db
-rw-r--r-- 1 root root    723 Sep 25 11:56 derby.log
-rw-rw-r-- 1 root root  17069 Sep 25 11:56 PySpark_FileSystem_Partitioning.ipynb
-rw-rw-r-- 1 root root  14292 Sep 25 11:53 PySpark_Write_실습.ipynb
drwxrwxr-x 5 root root    160 Sep 25 11:53 wordcount
drwxr-xr-x 5 root root    160 Sep 25 11:52 dataOutput
-rw-rw-r-- 1 root root  31964 Jul 24 04:29 PySpark_ML_실습1.ipynb
-rw-rw-r-- 1 root root  29648 Jul 24 04:29 PySpark_ML_실습2.ipynb
-rw-rw-r-- 1 root root  22327 Jul 24 04:29 PySpark_Schema_Evolution.ipynb
-rw-rw-r-- 1 root root   1593 Jul 24 04:29 bucketDemo.py
drwxrwxr-x 6 root root    192 Jul 24 04:29 join_broadcast_join
-rw-r--r-- 1 root root 143130 Jul  6  2018 appl_stock.csv
-rw-r--r-- 1 root root 143130 Jul  6  2018 appl_stock.csv.1


In [11]:
!ls -tl spark-warehouse/

total 4
drwxr-xr-x 9 root root 4096 Jan 29 04:27 appl_stock


In [9]:
!ls -tl spark-warehouse/appl_stock/

total 0
-rw-r--r--  1 root root   0 Sep 25 11:57  _SUCCESS
drwxr-xr-x 14 root root 448 Sep 25 11:57 'year=2016'
drwxr-xr-x 14 root root 448 Sep 25 11:57 'year=2015'
drwxr-xr-x 14 root root 448 Sep 25 11:57 'year=2014'
drwxr-xr-x 14 root root 448 Sep 25 11:57 'year=2013'
drwxr-xr-x 14 root root 448 Sep 25 11:57 'year=2012'
drwxr-xr-x 14 root root 448 Sep 25 11:57 'year=2011'
drwxr-xr-x 14 root root 448 Sep 25 11:57 'year=2010'


In [10]:
!ls -tl spark-warehouse/appl_stock/year\=2010/

total 0
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=12'
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=11'
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=10'
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=9'
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=8'
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=7'
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=6'
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=5'
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=4'
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=3'
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=2'
drwxr-xr-x 4 root root 128 Sep 25 11:57 'month=1'


In [14]:
!ls -tl spark-warehouse/appl_stock/year\=2010/month\=12/

total 4
-rw-r--r-- 1 root root 3027 Jan 29 04:27 part-00000-ee4ef65d-6e86-4c64-8880-aa44a099aad9.c000.snappy.parquet


### How to Read from Partitioned table

In [13]:
df = spark.read.table("appl_stock").where("year = 2016 and month = 12")

In [14]:
df.show(10)

+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|               Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01 00:00:00|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02 00:00:00|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05 00:00:00|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06 00:00:00|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07 00:00:00|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08 00:00:00|        110

In [15]:
spark.sql("SELECT * FROM appl_stock WHERE year = 2016 and month = 12").show(10)
input("wait..")

+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|               Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01 00:00:00|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02 00:00:00|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05 00:00:00|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06 00:00:00|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07 00:00:00|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08 00:00:00|        110

KeyboardInterrupt: Interrupted by user